In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/SMU_MITB_NLP/Group Project/NLP-Lyric-Generator/src/bin

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/SMU_MITB_NLP/Group Project/NLP-Lyric-Generator/src/bin


In [ ]:
### Standard Imports
import numpy as np
import re
import os
import sys
from collections import Counter

import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
### Custom Imports
sys.path.append('../')
import lib.utilities as utils

In [ ]:
### Text Parameters
start_token = '<cls>'
end_token = '<eos>'
pad_token = '<pad>'

### General Parameters
random_seed = 2022
model_folder = '../../models/autoencoder/lstm/v3'

### Model Parameters
val_split = 0.2
window_len = 15
batch_size = 64
embedding_dim = 128
enc_dim, dec_dim = 256, 256
learn_rate = 0.001
epochs = 80
dropout = 0.05
recurrent_dropout = 0.05

In [ ]:
### Load Data
corpus = utils.load_corpus()

In [ ]:
### Pre-Processing Text
words = utils.preprocess_text(corpus, fun_list = [utils.to_lower, utils.decontraction, utils.remove_punct], keep = '\<|\>')
words = re.sub('\n',' \n ', words)
words = re.split(' +', words) #Tokenising

word_count = Counter(words) # Assumes end_token is already in the corpus
word_count[start_token] = 0
word_count[pad_token] = 0

#Reference Dictionaries to convert one-hot index to string and vice versa
index_to_vocab = {i: k for i, k in enumerate(word_count.keys())}
vocab_to_index = {k: i for i, k in enumerate(word_count.keys())}

songs = ' '.join(words)
songs = songs.split(' \n \n <eos> \n \n ')
songs = [song.split(' ') for song in songs]
songs = [[pad_token]*(window_len-1) + [start_token] + song + [end_token] + [pad_token]*(window_len-1) for song in songs]
songs_token_ind = [[vocab_to_index.get(x) for x in song] for song in songs]

In [ ]:
### Creating Dataset
x_encoder = []
x_decoder = []
y = []
vocab_size = len(word_count)

for song in songs_token_ind:
  for i in range(len(song)-window_len):
    x_encoder.append(song[(i+1):(i+window_len+1)])
    x_decoder.append(song[i:(i+window_len)])
    y.append(song[i+window_len])

In [ ]:
rand_int = np.random.randint(0, len(x_encoder), 1)[0]
print([index_to_vocab.get(x) for x in x_encoder[rand_int]])
print([index_to_vocab.get(x) for x in x_decoder[rand_int]])
print(index_to_vocab.get(y[rand_int]))

['future', '\n', 'together', 'you', 'and', 'i', 'we', 'are', '\n', '\n', '<chorus>', '\n', 'one', 'singapore', '\n']
['brighter', 'future', '\n', 'together', 'you', 'and', 'i', 'we', 'are', '\n', '\n', '<chorus>', '\n', 'one', 'singapore']




In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(((x_encoder, x_decoder), y))
dataset = dataset.shuffle(buffer_size = 10000, seed = random_seed)
train_dataset = dataset.take(int((1-val_split)*len(dataset)))
val_dataset = dataset.skip(int((1-val_split)*len(dataset)))

train_dataset_final = train_dataset.batch(batch_size).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_dataset_final = val_dataset.batch(batch_size).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# Encoder
encoder_input = layers.Input(shape=(window_len), name = 'encoder_input')
encoder_embedded = layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length = window_len, name = 'encoder_embedding')(
    encoder_input
)

# Return state in addition to output
encoder_output, encoder_hidden_state, encoder_cell_state = layers.LSTM(enc_dim,
                                                                       dropout = dropout, recurrent_dropout = recurrent_dropout,
                                                                       return_state=True, name = "encoder_lstm")(encoder_embedded)

# Decoder
decoder_input = layers.Input(shape=(window_len), name = 'decoder_input')
decoder_embedded = layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length = window_len, name = 'decoder_embedding')(
    decoder_input
)

# Pass the encoder state to a new LSTM, as initial state
decoder_output = layers.LSTM(dec_dim,
                             dropout = dropout, recurrent_dropout = recurrent_dropout,
                             name="decoder_lstm")(decoder_embedded, initial_state=[encoder_hidden_state, encoder_cell_state])

# Attention
attention_context_vector = tf.keras.layers.Attention(name = 'attention')(inputs = [decoder_output, encoder_output])

# Output
output = layers.Dense(vocab_size, name = 'output', activation = 'softmax')(tf.concat([decoder_output, attention_context_vector], 1))

model = tf.keras.Model((encoder_input, decoder_input), output, name = 'LSTM Encoder-Decoder with Dense Embeddings and Attention')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input (InputLayer)     [(None, 15, 1049)]   0           []                               
                                                                                                  
 decoder_input (InputLayer)     [(None, 15, 1049)]   0           []                               
                                                                                                  
 encoder_lstm (LSTM)            [(None, 256),        1337344     ['encoder_input[0][0]']          
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                            

In [ ]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate),
              metrics = ['accuracy'])

In [ ]:
### Callbacks
callback_es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=5,
    verbose=1,
    mode='min',
    baseline=None,
    restore_best_weights=True
)

callback_mc = tf.keras.callbacks.ModelCheckpoint(
    filepath='weights.{epoch:02d}-{val_loss:.2f}-{val_accuracy:.2f}.hdf5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

In [ ]:
history = model.fit(x = train_dataset_final, validation_data = val_dataset_final, epochs = epochs, callbacks = [callback_es, callback_mc])

Epoch 1/80
181/181 [==============================] - 133s 705ms/step - loss: 5.0912 - accuracy: 0.1840 - val_loss: 4.6535 - val_accuracy: 0.2197
Epoch 2/80
181/181 [==============================] - 124s 684ms/step - loss: 4.2379 - accuracy: 0.2520 - val_loss: 3.5873 - val_accuracy: 0.2997
Epoch 3/80
181/181 [==============================] - 126s 694ms/step - loss: 3.0037 - accuracy: 0.4063 - val_loss: 2.2561 - val_accuracy: 0.5703
Epoch 4/80
181/181 [==============================] - 124s 686ms/step - loss: 2.0851 - accuracy: 0.5957 - val_loss: 1.6548 - val_accuracy: 0.6895
Epoch 5/80
181/181 [==============================] - 126s 697ms/step - loss: 1.6361 - accuracy: 0.6735 - val_loss: 1.3503 - val_accuracy: 0.7266
Epoch 6/80
181/181 [==============================] - 126s 699ms/step - loss: 1.3772 - accuracy: 0.7214 - val_loss: 1.2353 - val_accuracy: 0.7495
Epoch 7/80
181/181 [==============================] - 126s 695ms/step - loss: 1.2427 - accuracy: 0.7422 - val_loss: 1.1605 -

In [ ]:
model.save_weights(model_folder)

In [ ]:
def generate_text(model, start_string, num_generate = 1000, temperature=1.0, random_seed = random_seed):
    # Converting our start string to numbers (vectorizing).
    input_indices = [vocab_to_index.get(s) for i, s in enumerate(start_string) if i < window_len-1]
    input_indices = [vocab_to_index.get(pad_token)]*(window_len - len(input_indices)-1) + [vocab_to_index.get(start_token)] + input_indices

    input_oh = tf.one_hot(input_indices, depth = vocab_size)
    x = tf.expand_dims(input_oh, 0)

    # Empty string to store our results.
    text_generated = []

    # Here batch size == 1.
    model.reset_states()
    for word_index in range(num_generate):
        prediction = model.predict([x,x])

        # Using a categorical distribution to predict the character returned by the model.
        prediction = prediction / temperature
        predicted_id = tf.random.categorical(prediction, num_samples=1, seed = random_seed)[-1,0]
        predicted_oh = tf.one_hot(predicted_id, depth = vocab_size)

        # We pass the series of previous words (up to window length) as the next input to the model
        # along with the previous hidden state.
        input_index = tf.expand_dims([predicted_oh], 0)
        x = tf.concat([x[:,1:,:],input_index], 1)
        
        pred_word = index_to_vocab[predicted_id.numpy()]
        text_generated.append(pred_word)
        if pred_word == end_token:
          break
    
    return (' '.join(start_string) + ' ' + ' '.join(text_generated)), text_generated

In [ ]:
result_str, result = generate_text(model, start_string=['<verse>','\n','step','by','step'], num_generate=50, temperature=1.0)

In [ ]:
print(result_str)